In [24]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
from scipy.signal import chirp, find_peaks, peak_widths

import warnings
warnings.filterwarnings('ignore')

In [25]:
# prep info
# contact: c
# status inlcude: r, s, a

root_path = r'/Users/xiaoqiansun/Desktop/MedLu/TubeTest/Data'


days = 6
fpsN = 15
fpsB = 30
baselinePeriod = 180 #(3min(180s) baseline)
redundentTime = 190 #10s+3min


M32_list = ['32-1_28', '32-2_18', '32-3_15', '32-4_20', '32-5_12', '32-6_15']
M33_list = ['33-1_28', '33-2_35', '33-3_32', '33-4_30', '33-5_30', '33-6_26']



# Define functions

In [26]:
# to keep certain status frrame interval
#----------------------------------------------------------------------------------
def pick_BInterval(m1_tube, m2_tube, s1, s2, c=None):      
      
    df1 = m1_tube.copy()
    df2 = m2_tube.copy()
    
    if c:
        df1 = df1[(df1['mouse1']== s1) & (df1['mouse2']== s2) & (df1['contact']== c)]
        df2 = df2[(df2['mouse1']== s1) & (df2['mouse2']== s2) & (df2['contact']== c)]
    
    else:
        df1 = df1[(df1['mouse1']== s1) & (df1['mouse2']== s2)]
        df2 = df2[(df2['mouse1']== s1) & (df2['mouse2']== s2)]
    
        
    df1 = df1.drop(['mouse1', 'mouse2', 'contact'], axis=1)
    df2 = df2.drop(['mouse1', 'mouse2', 'contact'], axis=1)
    
    return(df1, df2)

In [27]:
# calculate event rate&amplitude
#----------------------------------------------------------------------------------
def event_rate(df, fps):
        
    # caculate event rate
    Frames, n_neuron = df.shape
    # get array
    df_array = df.to_numpy()
    
    peak_sum = 0
    amplitude_mean_list = []
    amplitude_mean_list_wit_zero = []
    
    for i in range(n_neuron):
        peaks, _ = find_peaks(df_array[:,i],  prominence=2.5, width=2)
        peak_sum = peak_sum + len(peaks)
        
        if len(peaks) != 0:
            amplitude_mean = sum(df_array[:,i][peaks])/len(peaks)
            amplitude_mean_list.append(amplitude_mean)
            amplitude_mean_list_wit_zero.append(amplitude_mean)
        else:
            amplitude_mean_list_wit_zero.append(0)

    # event_rate: # of peaks per second per neuron
    if Frames != 0 and n_neuron != 0:
        event_rate = peak_sum/(Frames/fps)/n_neuron
    else:
        event_rate = np.nan
        
    # calculate amplitude mean for all neruons
    amplitude = np.mean(amplitude_mean_list)
    amplitude_00 = np.mean(amplitude_mean_list_wit_zero)
    
    #print('total peaks %i'%peak_sum)
    return(event_rate, amplitude, amplitude_00)




# Call - TubeTest & Baseline

In [28]:
ER_32 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
ER_33 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM_32 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM_33 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                      'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM00_32 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                        'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})
AM00_33 = pd.DataFrame({'Days':np.arange(1,7), 'as':np.nan, 'sa': np.nan, 'rs':np.nan, 'sr': np.nan,
                        'ar':np.nan, 'ra': np.nan,'ss':np.nan, 'ssc': np.nan,})

Summary_s = [ER_32, AM_32, AM00_32, ER_33, AM_33, AM00_33]
Sheets_s = ['ER_32', 'AM_32', 'AM00_32', 'ER_33', 'AM_33', 'AM00_33']

for day in range(days):
    
    # read in dataframe
    m32_folder = os.path.join(root_path, M32_list[day])
    m33_folder = os.path.join(root_path, M33_list[day])
    
    
    m32_tube = pd.read_csv(os.path.join(m32_folder,'m32_tube.csv'), index_col = 'Frame')
    m33_tube = pd.read_csv(os.path.join(m33_folder,'m33_tube.csv'), index_col = 'Frame')     
    
    
    # separate sessions
    as1, as2 = pick_BInterval(m32_tube, m33_tube, 'a', 's', c=None)
    sa1, sa2 = pick_BInterval(m32_tube, m33_tube, 's', 'a', c=None)
    rs1, rs2 = pick_BInterval(m32_tube, m33_tube, 'r', 's', c=None)
    sr1, sr2 = pick_BInterval(m32_tube, m33_tube, 's', 'r', c=None)
    ar1, ar2 = pick_BInterval(m32_tube, m33_tube, 'a', 'r', c=None)
    ra1, ra2 = pick_BInterval(m32_tube, m33_tube, 'r', 'a', c=None)
    
    ss1, ss2 = pick_BInterval(m32_tube, m33_tube, 's', 's', c='nc')
    ssc1, ssc2 = pick_BInterval(m32_tube, m33_tube, 's', 's', c='c')
    
    
    # calculate ER, AM, AM00
    M32_dfs = [as1,sa1,rs1,sr1,ar1,ra1,ss1,ssc1]
    M33_dfs = [as2,sa2,rs2,sr2,ar2,ra2,ss2,ssc2]
    
    er_32s = []; am_32s = []; am00_32s = []
    er_33s = []; am_33s = []; am00_33s = []
    ss = [er_32s,am_32s,am00_32s,er_33s,am_33s,am00_33s]
    for i in range(len(M32_dfs)):
        
        er32, am32, am0032 = event_rate(M32_dfs[i], fpsN)
        er33, am33, am0033 = event_rate(M33_dfs[i], fpsN)
        er_32s.append(er32); am_32s.append(am32); am00_32s.append(am0032)
        er_33s.append(er33); am_33s.append(am33); am00_33s.append(am0033)
        
    
    # update into dataframe
    for m in range(len(Summary_s)):
        Summary_s[m].loc[day, ER_32.columns[1:]] = ss[m]
    
    
    
writer = pd.ExcelWriter('Summary.xlsx')
for k in range(len(Summary_s)):
    Summary_s[k].to_excel(writer, sheet_name=Sheets_s[k])


In [29]:
ERB_32 = pd.DataFrame({'Days':np.arange(1,7), 'ER':np.nan, 'AM': np.nan, 'AM00':np.nan})
ERB_33 = pd.DataFrame({'Days':np.arange(1,7), 'ER':np.nan, 'AM': np.nan, 'AM00':np.nan})

erB_32s = []; amB_32s = []; am00B_32s = []
erB_33s = []; amB_33s = []; am00B_33s = []

for day in range(days):
    
    m32_folder = os.path.join(root_path, M32_list[day])
    m33_folder = os.path.join(root_path, M33_list[day])
    
    
    m32_base = pd.read_csv(os.path.join(m32_folder,'m32_base.csv'), index_col = 'Frame')
    m33_base = pd.read_csv(os.path.join(m33_folder,'m33_base.csv'), index_col = 'Frame')
        
    er32, am32, am0032 = event_rate(m32_base, fpsN)
    er33, am33, am0033 = event_rate(m33_base, fpsN)
    
    erB_32s.append(er32); amB_32s.append(am32); am00B_32s.append(am0032)
    erB_33s.append(er33); amB_33s.append(am33); am00B_33s.append(am0033)
        
    
    for m in range(len(Summary_s)):
        Summary_s[m].loc[day, ER_32.columns[1:]] = ss[m]
    
    
ERB_32['ER'] = erB_32s; ERB_32['AM'] = amB_32s; ERB_32['AM00'] = am00B_32s
ERB_33['ER'] = erB_33s; ERB_33['AM'] = amB_33s; ERB_33['AM00'] = am00B_33s

ERB_32.to_excel(writer, sheet_name='Mouse 32 Baseline')
ERB_33.to_excel(writer, sheet_name='Mouse 33 Baseline')
writer.save()